# 🏠 House Price Prediction Project

Regression Model with OOP Structure & Pipelines

In [6]:

# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Metrics
from sklearn.metrics import mean_squared_error, r2_score

import pickle


## 1️⃣ Data Loader Class

In [7]:

class DataLoader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = None

    def load_data(self):
        self.data = pd.read_csv(self.filepath)
        return self.data


## 2️⃣ Data Preprocessor Class

In [8]:

class DataPreprocessor:
    def __init__(self, data, target_column):
        self.data = data
        self.target = target_column

    def preprocess(self):
        self.data[self.target] = np.log1p(self.data[self.target])

        X = self.data.drop(self.target, axis=1)
        y = self.data[self.target]

        numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
        categorical_cols = X.select_dtypes(include=['object']).columns

        numeric_pipeline = Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ])

        categorical_pipeline = Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("encoder", OneHotEncoder(handle_unknown="ignore"))
        ])

        preprocessor = ColumnTransformer([
            ("num", numeric_pipeline, numerical_cols),
            ("cat", categorical_pipeline, categorical_cols)
        ])

        return X, y, preprocessor


## 3️⃣ Model Trainer Class

In [9]:

class ModelTrainer:
    def __init__(self, preprocessor):
        self.preprocessor = preprocessor
        self.models = {
            "LinearRegression": LinearRegression(),
            "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42)
        }
        self.best_model = None

    def train_and_evaluate(self, X_train, X_test, y_train, y_test):
        best_rmse = float("inf")

        for name, model in self.models.items():
            pipeline = Pipeline([
                ("preprocessing", self.preprocessor),
                ("model", model)
            ])

            pipeline.fit(X_train, y_train)
            predictions = pipeline.predict(X_test)

            rmse = np.sqrt(mean_squared_error(y_test, predictions))
            r2 = r2_score(y_test, predictions)

            print(f"Model: {name}")
            print("RMSE:", rmse)
            print("R2 Score:", r2)
            print("-"*40)

            if rmse < best_rmse:
                best_rmse = rmse
                self.best_model = pipeline

        return self.best_model


## 4️⃣ Execute Full Pipeline

In [10]:

# Load Data
loader = DataLoader("train.csv")
data = loader.load_data()

# Preprocess
preprocessor_obj = DataPreprocessor(data, "SalePrice")
X, y, preprocessor = preprocessor_obj.preprocess()

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
trainer = ModelTrainer(preprocessor)
best_model = trainer.train_and_evaluate(X_train, X_test, y_train, y_test)

# Save Model
with open("house_price_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("✅ Model training complete and saved!")


Model: LinearRegression
RMSE: 0.1280051997532984
R2 Score: 0.9121952501952239
----------------------------------------
Model: RandomForest
RMSE: 0.14634504009071186
R2 Score: 0.8852325314758505
----------------------------------------
✅ Model training complete and saved!
